# Preparación

## Dependencias

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
pd.options.display.max_colwidth = 500
pd.options.display.max_columns = 50

## Helpers

In [309]:
def aggregate_by_col(tabla, col, values='ACCESOS'):
    return tabla.groupby(['ANNO', 'TRIMESTRE', col])[values].sum().reset_index().pivot_table(index=['ANNO', 'TRIMESTRE'], columns=col, values=values).fillna(0).astype(int)

In [322]:
def get_concentracion(tabla):
    
    tabla_p = tabla.div(tabla.sum(axis=1), axis=0)

    tabla_p = pd.concat([
        tabla_p.loc[(2017, 2)].sort_values(ascending=False),
        tabla_p.loc[(2022, 2)].sort_values(ascending=False)], axis=1)

    tabla_p.columns = ['porcentaje_2017', 'porcentaje_2022']
    tabla_p['porcentaje_diferencia'] = (tabla_p.porcentaje_2022 - tabla_p.porcentaje_2017)
    tabla_p[['porcentaje_cumulativo_2017', 'porcentaje_cumulativo_2022']] = tabla_p[['porcentaje_2017', 'porcentaje_2022']].cumsum()
    
    return tabla_p

# Internet fijo

## Fuentes

### Accesos de Internet Fijo desde 2017-2T

desde [postdata](https://postdata.gov.co/dataset/suscriptores-e-ingresos-de-internet-fijo/resource/540ea080-bf16-4d63-911f-3b4814e8e4f1#%7Bview-grid:%7BcolumnsWidth:%5B%7Bcolumn:!accesos,width:108%7D%5D%7D%7D)

In [2]:
internet_fijo = pd.read_csv('data/sources/internetfijo.csv', sep=';')

In [3]:
internet_fijo.shape

(1660574, 15)

In [86]:
internet_fijo.sample(1)

,ANNO,TRIMESTRE,ID_EMPRESA,EMPRESA,ID_MUNICIPIO,MUNICIPIO,ID_DEPARTAMENTO,DEPARTAMENTO,ID_SEGMENTO,SEGMENTO,VELOCIDAD_EFECTIVA_DOWNSTREAM,VELOCIDAD_EFECTIVA_UPSTREAM,ID_TECNOLOGIA,TECNOLOGIA,ACCESOS
37556,2017,2,900548102,AZTECA COMUNICACIONES COLOMBIA S.A.S,52110,BUESACO,52,NARIÑO,107,Corporativo,25,"12,5",108,Fiber to the home (FTTH),3


## Ideas

### Número de accesos de internet fijo por trimestre

In [148]:
fijo_accesos = internet_fijo.groupby(['ANNO', 'TRIMESTRE']).ACCESOS.sum()

In [281]:
fijo_accesos

ANNO  TRIMESTRE
2017  2            6193671
      3            6284505
      4            6352946
2018  1            6479592
      2            6578663
      3            6660066
      4            6731509
2019  1            6800865
      2            6911078
      3            7012936
      4            6982443
2020  1            7165355
      2            7490454
      3            7698521
      4            7835342
2021  1            8127334
      2            8292381
      3            8341313
      4            8444874
2022  1            8546208
      2            8482581
Name: ACCESOS, dtype: int64

In [149]:
fijo_accesos.to_csv('data/internetfijo_accesos.csv')

### Crecimiento trimestral en accesos de internet fijo

In [304]:
fijo_accesos_crecimiento = pd.concat([
    fijo_accesos.diff(),
    fijo_accesos.sub(fijo_accesos.iloc[0]),
    fijo_accesos.div(fijo_accesos.shift(1)),
    fijo_accesos.div(fijo_accesos.iloc[0])
], axis=1)

In [306]:
fijo_accesos_crecimiento.columns = ['diferencia trimestral', 'diferencia respecto a 2017-2T', 'crecimiento trimestral', 'crecimiento respecto a 2017-2T']

In [307]:
fijo_accesos_crecimiento

diferencia trimestral  diferencia respecto a 2017-2T  \
ANNO TRIMESTRE                                                         
2017 2                            NaN                              0   
     3                        90834.0                          90834   
     4                        68441.0                         159275   
2018 1                       126646.0                         285921   
     2                        99071.0                         384992   
     3                        81403.0                         466395   
     4                        71443.0                         537838   
2019 1                        69356.0                         607194   
     2                       110213.0                         717407   
     3                       101858.0                         819265   
     4                       -30493.0                         788772   
2020 1                       182912.0                         971684   
     2                       325099.0                        1296783   
     3                       208067.0                        1504850   
     4                       136821.0                        1641671   
2021 1                       291992.0                        1933663   
     2                       165047.0                        2098710   
     3                        48932.0                        2147642   
     4                       103561.0                        2251203   
2022 1                       101334.0                        2352537   
     2                       -63627.0                        2288910   

                crecimiento trimestral  crecimiento respecto a 2017-2T  
ANNO TRIMESTRE                                                          
2017 2                             NaN                        1.000000  
     3                        1.014666                        1.014666  
     4                        1.010890                        1.025716  
2018 1                        1.019935                        1.046163  
     2                        1.015290                        1.062159  
     3                        1.012374                        1.075302  
     4                        1.010727                        1.086837  
2019 1                        1.010303                        1.098035  
     2                        1.016206                        1.115829  
     3                        1.014738                        1.132275  
     4                        0.995652                        1.127351  
2020 1                        1.026196                        1.156883  
     2                        1.045371                        1.209372  
     3                        1.027778                        1.242966  
     4                        1.017772                        1.265056  
2021 1                        1.037266                        1.312200  
     2                        1.020308                        1.338848  
     3                        1.005901                        1.346748  
     4                        1.012415                        1.363468  
2022 1                        1.011999                        1.379829  
     2                        0.992555                        1.369556

In [308]:
fijo_accesos_crecimiento.to_csv('data/internetfijo_accesos_crecimento.csv')

### Número de accesos a internet fijo por departamento y trimestre

In [310]:
fijo_departamentos = aggregate_by_col(internet_fijo, 'ID_DEPARTAMENTO')

In [312]:
depto_dict = internet_fijo.groupby('ID_DEPARTAMENTO').DEPARTAMENTO.first().to_dict()

In [313]:
fijo_departamentos = fijo_departamentos.rename(columns=depto_dict)

In [319]:
fijo_departamentos = fijo_departamentos[fijo_departamentos.sum().sort_values(ascending=False).index]

In [320]:
fijo_departamentos

ID_DEPARTAMENTO  BOGOTÁ D.C.  ANTIOQUIA  VALLE DEL CAUCA  CUNDINAMARCA  \
ANNO TRIMESTRE                                                           
2017 2               1776759    1074066           685493        334699   
     3               1807827    1090819           698081        338335   
     4               1818094    1098843           707051        342365   
2018 1               1826350    1113650           719755        369021   
     2               1857309    1127759           731687        384610   
     3               1875396    1149057           741593        381712   
     4               1876271    1164147           749987        391605   
2019 1               1874506    1187759           763692        399818   
     2               1890765    1198415           768930        413239   
     3               1915409    1212047           779626        422699   
     4               1916910    1232910           782509        416552   
2020 1               1956766    1258347           796990        432868   
     2               2024199    1322906           827014        459422   
     3               2068167    1346629           850067        497834   
     4               2087437    1363693           864036        515765   
2021 1               2137106    1407864           879885        541850   
     2               2163410    1464234           885944        552263   
     3               2152436    1489033           886623        538237   
     4               2169165    1509713           892882        569267   
2022 1               2189547    1529908           889558        573187   
     2               2197597    1524474           907437        573672   

ID_DEPARTAMENTO  ATLÁNTICO  SANTANDER  BOLÍVAR  RISARALDA  TOLIMA  \
ANNO TRIMESTRE                                                      
2017 2              314991     326858   174788     145637  147894   
     3              322437     331507   174669     146913  140366   
     4              328389     329803   177184     149943  147084   
2018 1              335464     334159   180168     162303  152223   
     2              338205     318321   183550     165341  153989   
     3              343197     321125   183812     167548  152342   
     4              346822     324351   186967     168549  153980   
2019 1              329666     329265   191419     173184  157890   
     2              343938     330025   194076     175933  158875   
     3              351804     339143   198475     178076  161315   
     4              354675     328405   185788     178752  152485   
2020 1              362839     338856   191306     186033  156311   
     2              376574     350673   197383     195029  164286   
     3              385913     362089   198025     202089  166827   
     4              391057     366945   197095     210869  180695   
2021 1              407995     385250   207639     216726  196546   
     2              421947     390558   221551     218652  199172   
     3              433582     395228   229480     217055  198204   
     4              425338     405967   234193     221651  200141   
2022 1              436012     410953   236081     234270  196884   
     2              424788     413974   227879     221986  202960   

ID_DEPARTAMENTO  NORTE DE SANTANDER  ...  CHOCÓ  ARAUCA  PUTUMAYO  \
ANNO TRIMESTRE                       ...                            
2017 2                       126136  ...  18427    9242     10897   
     3                       128865  ...  18912    9580     11435   
     4                       129802  ...  20514   10647     12200   
2018 1                       134169  ...  21013   11105     12772   
     2                       137851  ...  21302   11263     13534   
     3                       142502  ...  22087   11260     14645   
     4                       143285  ...  21285   11272     15553   
2019 1                       143100  ...  23996   12948     11036   
     2             

In [321]:
fijo_departamentos.to_csv('data/internetfijo_accesos_por_departamento.csv')

### Porcentaje de accesos a internet fijo por departamento

In [325]:
fijo_departamentos_concentracion = get_concentracion(fijo_departamentos)

In [326]:
fijo_departamentos_concentracion

,porcentaje_2017,porcentaje_2022,porcentaje_diferencia,porcentaje_cumulativo_2017,porcentaje_cumulativo_2022
ID_DEPARTAMENTO,,,,,
BOGOTÁ D.C.,0.286867,0.259072,-0.027795,0.286867,0.259072
ANTIOQUIA,0.173413,0.179718,0.006305,0.460280,0.438790
VALLE DEL CAUCA,0.110676,0.106977,-0.003700,0.570957,0.545766
CUNDINAMARCA,0.054039,0.067629,0.013591,0.624996,0.613396
SANTANDER,0.052773,0.048803,-0.003970,0.677768,0.662199
ATLÁNTICO,0.050857,0.050078,-0.000779,0.728625,0.712276
BOLÍVAR,0.028220,0.026864,-0.001356,0.756846,0.739141
TOLIMA,0.023878,0.023927,0.000048,0.780724,0.763067
RISARALDA,0.023514,0.026170,0.002656,0.804238,0.789237


In [327]:
fijo_departamentos_p.to_csv('data/internetfijo_accesos_por_departamento_concentracion.csv')

### Crecimiento trimestral de accesos a internet fijo en cuartiles de departamentos

In [381]:
depto_quartiles = pd.qcut(fijo_departamentos.loc[(2022,2)], 4, labels=range(1,5)).to_dict()

In [408]:
fijo_departamentos_crecimiento_cuartil = fijo_departamentos.stack().reset_index(name='accesos')
fijo_departamentos_crecimiento_cuartil.ID_DEPARTAMENTO = fijo_departamentos_crecimiento_cuartil.ID_DEPARTAMENTO.map(depto_quartiles)
fijo_departamentos_crecimiento_cuartil = fijo_departamentos_crecimiento_cuartil.groupby(['ANNO', 'TRIMESTRE', 'ID_DEPARTAMENTO']).accesos.sum()
fijo_departamentos_crecimiento_cuartil = fijo_departamentos_crecimiento_cuartil.reset_index().pivot_table(index=['ANNO', 'TRIMESTRE'], columns='ID_DEPARTAMENTO', values='accesos')
fijo_departamentos_crecimiento_cuartil = fijo_departamentos_crecimiento_cuartil.diff().iloc[1:].astype(int)

In [409]:
fijo_departamentos_crecimiento_cuartil

ID_DEPARTAMENTO      1      2      3       4
ANNO TRIMESTRE                              
2017 3            1272   3103  16690   69769
     4            3256   8184   9199   47802
2018 1             749  13841  17719   94337
     2             763  10677  19953   67678
     3            1321  14312  10759   55011
     4            1484  12507  10555   46897
2019 1           -1171   6747  19260   44520
     2            7312  15301  20603   66997
     3           -4928   8159  14229   84398
     4            4364   1160 -26409   -9608
2020 1            -231  15269  36544  131330
     2            5997  32383  49549  237170
     3            6140   6844  34929  160154
     4            2390    129  34350   99952
2021 1            3921  42269  42533  203269
     2          -12746  13766  27157  136870
     3            1566   3064  22155   22147
     4             612 -10320  24830   88439
2022 1             260   1181  31810   68083
     2          -10804 -27270 -23920   -1633

In [407]:
fijo_departamentos_crecimiento_cuartil.to_csv('data/internetfijo_accesos_por_departamento_crecimiento_cuartiles.csv')

### Crecimiento de accesos a internet fijo por departamento

In [342]:
def get_crecimiento(tabla):
    
    tabla_c = pd.concat([tabla.loc[(2017, 2)], tabla.loc[(2022, 2)]], axis=1).sort_values((2022, 2), ascending=False)
    tabla_c.columns = ['2017', '2022']
    tabla_c = pd.concat([
        tabla_c,
        (tabla_c['2022'] - tabla_c['2017']).rename('diferencia'),
        (tabla_c['2022'] / tabla_c['2017']).rename('crecimiento')
    ], axis=1)
    tabla_c = tabla_c.sort_values(['diferencia'], ascending=False)
    return tabla_c

In [418]:
fijo_departamentos_crecimiento = get_crecimiento(fijo_departamentos)

In [419]:
fijo_departamentos_crecimiento.insert(0, 'cuartil_2022', [depto_quartiles[i] for i in fijo_departamentos_crecimiento.index])

In [420]:
fijo_departamentos_crecimiento

,cuartil_2022,2017,2022,diferencia,crecimiento
ID_DEPARTAMENTO,,,,,
ANTIOQUIA,4,1074066,1524474,450408,1.419349
BOGOTÁ D.C.,4,1776759,2197597,420838,1.236857
CUNDINAMARCA,4,334699,573672,238973,1.713994
VALLE DEL CAUCA,4,685493,907437,221944,1.323773
ATLÁNTICO,4,314991,424788,109797,1.348572
SANTANDER,4,326858,413974,87116,1.266526
RISARALDA,4,145637,221986,76349,1.524242
NORTE DE SANTANDER,3,126136,185299,59163,1.469041
TOLIMA,4,147894,202960,55066,1.372334


In [421]:
fijo_departamentos_crecimiento.to_csv('data/internetfijo_accesos_por_departamento_crecimiento.csv')

### Número de accesos por empresa y trimestre

In [259]:
fijo_por_empresa = internet_fijo.groupby(['ID_EMPRESA', 'ANNO', 'TRIMESTRE']).ACCESOS.sum().reset_index().pivot_table(index=['ANNO', 'TRIMESTRE'], columns='ID_EMPRESA', values='ACCESOS').fillna(0).astype(int)

In [260]:
emp_dict = internet_fijo.groupby('ID_EMPRESA').EMPRESA.first().to_dict()

In [263]:
fijo_por_empresa = fijo_por_empresa.rename(columns=emp_dict)

In [264]:
fijo_por_empresa = fijo_por_empresa[fijo_por_empresa.sum().sort_values(ascending=False).index]

In [265]:
fijo_por_empresa

ID_EMPRESA      COMUNICACION CELULAR S A COMCEL S A  \
ANNO TRIMESTRE                                        
2017 2                                            0   
     3                                            0   
     4                                            0   
2018 1                                            0   
     2                                            0   
     3                                            0   
     4                                            0   
2019 1                                            0   
     2                                      2610782   
     3                                      2647664   
     4                                      2664110   
2020 1                                      2747552   
     2                                      2879580   
     3                                      3005507   
     4                                      3065976   
2021 1                                      3175493   
     2                                      3209411   
     3                                      3180321   
     4                                      3231312   
2022 1                                      3250531   
     2                                      3242542   

ID_EMPRESA      UNE EPM TELECOMUNICACIONES S.A.  \
ANNO TRIMESTRE                                    
2017 2                                  1250018   
     3                                  1275003   
     4                                  1278451   
2018 1                                  1311196   
     2                                  1325817   
     3                                  1354791   
     4                                  1382814   
2019 1                                  1415373   
     2                                  1442224   
     3                                  1474656   
     4                                  1506821   
2020 1                                  1517996   
     2                                  1594990   
     3                                  1594920   
     4                                  1618325   
2021 1                                  1657301   
     2                                  1686313   
     3                                  1714644   
     4                                  1733882   
2022 1                                  1762784   
     2                                  1776438   

ID_EMPRESA      COLOMBIA TELECOMUNICACIONES S.A. E.S.P.  TELMEX COLOMBIA S.A.  \
ANNO TRIMESTRE                                                                  
2017 2                                           966600               2278348   
     3                                           968908               2313890   
     4                                           967879               2349281   
2018 1                                           986358               2384831   
     2                                          1001954               2423948   
     3                                          1008300               2465033   
     4                                          1002199               2499415   
2019 1                                           997198               2562264   
     2                                           984774                     0   
     3                                           992369                     0   
     4                                           994351                     0   
2020 1                                          1005305                     0   
     2                                          1184090                     0   
     3                                          1172659                     0   
     4                                          1156342                     0   
2021 1                                          1168215                     0   
     2                                          1165912                     0   
     3               

In [142]:
fijo_por_empresa.to_csv('data/internetfijo_accesos_por_empresa.csv')

### Porcentaje de accesos a internet fijo por empresa en 2022

In [445]:
fijo_empresa_concentracion = pd.DataFrame(fijo_por_empresa.loc[(2022,2)].sort_values(ascending=False).rename('accesos_2022'))
fijo_empresa_concentracion['porcentaje'] = fijo_empresa_concentracion.accesos_2022 / fijo_empresa_concentracion.accesos_2022.sum()
fijo_empresa_concentracion['porcentaje_cumulativo'] = fijo_empresa_concentracion['porcentaje'].cumsum()

In [466]:
fijo_empresa_concentracion

,accesos_2022,porcentaje,porcentaje_cumulativo
ID_EMPRESA,,,
COMUNICACION CELULAR S A COMCEL S A,3242542,0.382259,0.382259
UNE EPM TELECOMUNICACIONES S.A.,1776438,0.209422,0.591681
COLOMBIA TELECOMUNICACIONES S.A. E.S.P.,1230778,0.145095,0.736776
EMPRESA DE TELECOMUNICACIONES DE BOGOTA S.A. ESP,684980,0.080751,0.817527
DIRECTV COLOMBIA LTDA,158141,0.018643,0.836170
...,...,...,...
LIKES.NET S.A.S,0,0.000000,1.000000
ISP TECHNOLOGY SAS,0,0.000000,1.000000
SISTELINTER S.A.S,0,0.000000,1.000000


In [456]:
fijo_empresa_concentracion.to_csv('data/internetfijo_accesos_por_empresa_concentracion.csv')

In [465]:
fijo_por_empresa[fijo_empresa_concentracion.head(12).index].to_csv('data/internetfijo_accesos_por_empresa_top12.csv')

### Porcentaje de accesos a internet fijo por empresa en 2017

In [453]:
fijo_empresa_concentracion0 = pd.DataFrame(fijo_por_empresa.loc[(2017,2)].sort_values(ascending=False).rename('accesos_2017'))
fijo_empresa_concentracion0['porcentaje'] = fijo_empresa_concentracion0.accesos_2017 / fijo_empresa_concentracion0.accesos_2017.sum()
fijo_empresa_concentracion0['porcentaje_cumulativo'] = fijo_empresa_concentracion0['porcentaje'].cumsum()

In [454]:
fijo_empresa_concentracion0.head(20)

,accesos_2017,porcentaje,porcentaje_cumulativo
ID_EMPRESA,,,
TELMEX COLOMBIA S.A.,2278348,0.367851,0.367851
UNE EPM TELECOMUNICACIONES S.A.,1250018,0.201822,0.569673
COLOMBIA TELECOMUNICACIONES S.A. E.S.P.,966600,0.156063,0.725735
EMPRESA DE TELECOMUNICACIONES DE BOGOTA S.A. ESP,643572,0.103908,0.829643
EDATEL S.A.,201142,0.032475,0.862119
AZTECA COMUNICACIONES COLOMBIA S.A.S,166025,0.026806,0.888924
DIRECTV COLOMBIA LTDA,126501,0.020424,0.909349
EMPRESA DE TELECOMUNICACIONES DE BUCARAMANGA S.A. E.S.P.,125980,0.020340,0.929689
METROTEL S.A,120576,0.019468,0.949156


In [457]:
fijo_empresa_concentracion0.to_csv('data/internetfijo_accesos_por_empresa_concentracion_2017.csv')

### Número de accesos a internet fijo por segmento y trimestre

In [469]:
fijo_segmentos = aggregate_by_col(internet_fijo, 'SEGMENTO')

In [470]:
fijo_segmentos

SEGMENTO        Corporativo  Residencial - Estrato 1  Residencial - Estrato 2  \
ANNO TRIMESTRE                                                                  
2017 2               543301                   627568                  2150384   
     3               548500                   618579                  2194890   
     4               553361                   687170                  2164659   
2018 1               552284                   637444                  2283184   
     2               557968                   645264                  2335155   
     3               564914                   649705                  2370713   
     4               586663                   668447                  2348570   
2019 1               565426                   685221                  2426176   
     2               572870                   738404                  2443491   
     3               582782                   754212                  2465173   
     4               588049                   666655                  2462577   
2020 1               591089                   707079                  2521295   
     2               576624                   760537                  2707218   
     3               557203                   816828                  2829020   
     4               564238                   852702                  2869772   
2021 1               573557                   907113                  2994041   
     2               589992                   984806                  3020039   
     3               591077                   977590                  3026424   
     4               625753                   977291                  3045757   
2022 1               637775                  1021359                  3015840   
     2               644795                   921464                  2987883   

SEGMENTO        Residencial - Estrato 3  Residencial - Estrato 4  \
ANNO TRIMESTRE                                                     
2017 2                          1769908                   677615   
     3                          1786154                   704760   
     4                          1796065                   711083   
2018 1                          1816161                   731824   
     2                          1821173                   743028   
     3                          1843357                   762464   
     4                          1854413                   778737   
2019 1                          1868931                   792458   
     2                          1880154                   798939   
     3                          1919776                   815037   
     4                          1955641                   831580   
2020 1                          2010906                   845055   
     2                          2085112                   860271   
     3                          2131646                   857726   
     4                          2165894                   863603   
2021 1                          2242305                   873832   
     2                          2264483                   874735   
     3                          2269356                   875659   
     4                          2306577                   883244   
2022 1                          2340604                   904178   
     2                          2402403                   910207   

SEGMENTO        Residencial - Estrato 5  Residencial - Estrato 6  \
ANNO TRIMESTRE                                                     
2017 2                           228633                   167010   
     3                           228289                   168601   
     4                           232036                   169578   
2018 1                           240024                   171966   
     2                           243523                   174089   
     3                           245494                   177098   
     4                     

In [471]:
fijo_segmentos.to_csv('data/internetfijo_accesos_por_segmento.csv')

### Porcentaje de accesos a internet fijo por segmento de usuarios y trimestre

In [474]:
fijo_segmentos_porcentaje = fijo_segmentos.div(fijo_segmentos.sum(axis=1), axis=0) * 100

In [475]:
fijo_segmentos_porcentaje

SEGMENTO        Corporativo  Residencial - Estrato 1  Residencial - Estrato 2  \
ANNO TRIMESTRE                                                                  
2017 2             8.771874                10.132408                34.719054   
     3             8.727815                 9.842923                34.925424   
     4             8.710305                10.816557                34.073310   
2018 1             8.523438                 9.837718                35.236540   
     2             8.481480                 9.808437                35.495890   
     3             8.482108                 9.755234                35.595939   
     4             8.715178                 9.930121                34.889205   
2019 1             8.314031                10.075498                35.674521   
     2             8.289155                10.684353                35.356148   
     3             8.310100                10.754583                35.151797   
     4             8.421823                 9.547590                35.268129   
2020 1             8.249263                 9.868025                35.187301   
     2             7.698118                10.153417                36.142242   
     3             7.237793                10.610194                36.747578   
     4             7.201192                10.882767                36.625995   
2021 1             7.057136                11.161261                36.839153   
     2             7.114868                11.876034                36.419443   
     3             7.086139                11.719858                36.282345   
     4             7.409856                11.572594                36.066340   
2022 1             7.462666                11.951020                35.288633   
     2             7.601401                10.863014                35.223749   

SEGMENTO        Residencial - Estrato 3  Residencial - Estrato 4  \
ANNO TRIMESTRE                                                     
2017 2                        28.576074                10.940442   
     3                        28.421554                11.214248   
     4                        28.271372                11.192965   
2018 1                        28.028941                11.294291   
     2                        27.683026                11.294514   
     3                        27.677759                11.448295   
     4                        27.548251                11.568535   
2019 1                        27.480784                11.652312   
     2                        27.204931                11.560266   
     3                        27.374783                11.621908   
     4                        28.007977                11.909585   
2020 1                        28.064290                11.793624   
     2                        27.836924                11.484898   
     3                        27.689033                11.141439   
     4                        27.642622                11.021893   
2021 1                        27.589675                10.751767   
     2                        27.307995                10.548659   
     3                        27.206220                10.497856   
     4                        27.313338                10.458936   
2022 1                        27.387632                10.579874   
     2                        28.321604                10.730307   

SEGMENTO        Residencial - Estrato 5  Residencial - Estrato 6  \
ANNO TRIMESTRE                                                     
2017 2                         3.691397                 2.696462   
     3                         3.632569                 2.682805   
     4                         3.652416                 2.669281   
2018 1                         3.704307                 2.653963   
     2                         3.701710                 2.646267   
     3                         3.686060                 2.659103   
     4                     

In [478]:
fijo_segmentos_porcentaje[['Residencial - Estrato {}'.format(i) for i in range(1,4)]].sum(axis=1)

ANNO  TRIMESTRE
2017  2            73.427536
      3            73.189901
      4            73.161239
2018  1            73.103198
      2            72.987353
      3            73.028931
      4            72.367578
2019  1            73.230802
      2            73.245433
      3            73.281162
      4            72.823695
2020  1            73.119615
      2            74.132583
      3            75.046804
      4            75.151385
2021  1            75.590089
      2            75.603473
      3            75.208423
      4            74.952273
2022  1            74.627285
      2            74.408367
dtype: float64

In [479]:
fijo_segmentos_porcentaje[['Residencial - Estrato {}'.format(i) for i in range(4,7)]].sum(axis=1)

ANNO  TRIMESTRE
2017  2            17.328302
      3            17.529622
      4            17.514662
2018  1            17.652562
      2            17.642491
      3            17.793457
      4            18.121999
2019  1            18.004577
      2            17.763683
      3            17.840987
      4            18.170159
2020  1            17.948029
      2            17.403338
      3            16.947489
      4            16.815156
2021  1            16.485750
      2            16.217417
      3            16.270520
      4            16.191775
2022  1            16.316652
      2            16.516282
dtype: float64

In [473]:
fijo_segmentos.div(fijo_segmentos.sum(axis=1), axis=0).to_csv('data/internetfijo_accesos_por_segmento_porcentaje.csv')

Número de accesos residenciales según estrato socioeconómico

In [87]:
fijo_residencial_por_segmento = internet_fijo[internet_fijo.SEGMENTO.str.contains('Residencial|Sin')][['ANNO', 'TRIMESTRE', 'SEGMENTO', 'ACCESOS']]

In [90]:
fijo_residencial_por_segmento.SEGMENTO = fijo_residencial_por_segmento.SEGMENTO.apply(lambda x: x[-1] if 'Residencial' in x else 0).astype(int)

In [91]:
fijo_residencial_por_segmento = fijo_residencial_por_segmento.groupby(['ANNO', 'TRIMESTRE', 'SEGMENTO']).ACCESOS.sum().reset_index().pivot_table(index=['ANNO', 'TRIMESTRE'], columns='SEGMENTO', values='ACCESOS')

In [92]:
fijo_residencial_por_segmento

SEGMENTO             0        1        2        3       4       5       6
ANNO TRIMESTRE                                                           
2017 2           29252   627568  2150384  1769908  677615  228633  167010
     3           34732   618579  2194890  1786154  704760  228289  168601
     4           38994   687170  2164659  1796065  711083  232036  169578
2018 1           46705   637444  2283184  1816161  731824  240024  171966
     2           58463   645264  2335155  1821173  743028  243523  174089
     3           46321   649705  2370713  1843357  762464  245494  177098
     4           53532   668447  2348570  1854413  778737  255108  186039
2019 1           30644   685221  2426176  1868931  792458  246958  185051
     2           48497   738404  2443491  1880154  798939  246541  182182
     3           39816   754212  2465173  1919776  815037  249259  186881
     4           40800   666655  2462577  1955641  831580  249125  188016
2020 1           48946   707079  2521295  2010906  845055  249547  191438
     2           57374   760537  2707218  2085112  860271  250953  192365
     3           59118   816828  2829020  2131646  857726  252530  194450
     4           65211   852702  2869772  2165894  863603  255215  198707
2021 1           70466   907113  2994041  2242305  873832  261438  204582
     2           88251   984806  3020039  2264483  874735  263371  206704
     3          119691   977590  3026424  2269356  875659  264698  216818
     4          122121   977291  3045757  2306577  883244  268936  215195
2022 1          136175  1021359  3015840  2340604  904178  274351  215926
     2          121454   921464  2987883  2402403  910207  278635  212165

Sería útil comparar estos valores con el número de hogares según estrato o el número de personas que habitan hogares por estrato para tener una idea del nivel de acceso a telecomunicaciones. No encuentro este dato.

### Velocidad efectiva de internet fijo por trimestre

In [129]:
fijo_velocidad_down = internet_fijo.groupby(['ANNO', 'TRIMESTRE'])['VELOCIDAD_EFECTIVA_DOWNSTREAM']
fijo_velocidad_up = internet_fijo.groupby(['ANNO', 'TRIMESTRE'])['VELOCIDAD_EFECTIVA_UPSTREAM']
fijo_velocidad_down, fijo_velocidad_up = [
    pd.concat([f.median().rename('median'), f.describe()[['min', '25%', '50%', '75%', 'max', 'mean', 'std']]], axis=1) for f in [fijo_velocidad_down, fijo_velocidad_up]
]

In [130]:
fijo_velocidad_down

median    min    25%    50%     75%        max        mean  \
ANNO TRIMESTRE                                                               
2017 2            5.00  0.009  2.000   5.00   10.00   204800.0   67.628981   
     3            5.00  0.009  2.000   5.00   10.00   204800.0   64.999430   
     4            5.00  0.009  2.000   5.00   10.00   204800.0   96.344426   
2018 1            5.12  0.009  2.048   5.12   10.00    50000.0   35.452908   
     2            6.00  0.009  2.500   6.00   10.24    30720.0   27.624643   
     3            6.00  0.009  3.000   6.00   12.00    50000.0   55.233938   
     4            6.00  0.009  3.000   6.00   14.00    50000.0   46.348453   
2019 1            6.00  0.009  3.000   6.00   15.00   122880.0   27.892722   
     2            8.00  0.009  4.000   8.00   20.00    50000.0   47.620466   
     3            8.00  0.009  3.000   8.00   20.00   409600.0   56.042497   
     4            9.00  0.009  4.000   9.00   20.00   409600.0   41.228925   
2020 1           10.00  0.003  4.000  10.00   20.00   716800.0   54.438478   
     2           10.00  0.003  4.000  10.00   22.00   512000.0  170.042358   
     3           10.00  0.003  4.100  10.00   25.00    50000.0   61.551479   
     4           10.00  0.003  5.000  10.00   30.00   100000.0   54.680532   
2021 1           10.00  0.003  5.000  10.00   30.00  2204800.0   83.557704   
     2           10.24  0.003  5.000  10.24   30.00  2204800.0   96.895239   
     3           12.00  0.010  5.120  12.00   40.00  2204800.0  123.710493   
     4           13.00  0.003  6.000  13.00   50.00  2204800.0  148.073308   
2022 1           15.00  0.003  6.000  15.00   50.00   310000.0   96.433428   
     2           20.00  0.000  6.000  20.00  100.00  3450300.0  190.391796   

                        std  
ANNO TRIMESTRE               
2017 2          1507.933459  
     3          1574.002825  
     4          1504.535982  
2018 1           491.471883  
     2           338.957006  
     3           776.193489  
     4           644.829783  
2019 1           571.099646  
     2           626.484777  
     3          1605.535365  
     4          1670.056647  
2020 1          2860.332336  
     2          3867.951949  
     3           682.308320  
     4           692.759296  
2021 1          7203.433775  
     2          7104.675474  
     3          8340.892500  
     4          9207.385039  
2022 1          1435.801755  
     2          9773.053848

In [146]:
fijo_velocidad_down.to_csv('data/internetfijo_velocidadefectiva_down.csv')

In [131]:
fijo_velocidad_up

median    min  25%   50%     75%        max        mean  \
ANNO TRIMESTRE                                                            
2017 2            1.00  0.009  0.8  1.00   2.000   204800.0   43.782694   
     3            1.00  0.019  1.0  1.00   2.048   204800.0   40.783730   
     4            1.00  0.008  1.0  1.00   2.500   204800.0   39.082905   
2018 1            1.00  0.008  1.0  1.00   3.000    20000.0   16.765971   
     2            1.00  0.019  1.0  1.00   3.000    20000.0   15.823991   
     3            1.00  0.019  1.0  1.00   4.000    40957.0   19.314962   
     4            1.00  0.008  1.0  1.00   4.000    20000.0   14.701498   
2019 1            1.00  0.008  1.0  1.00   4.000    55600.0   14.728370   
     2            1.02  0.008  1.0  1.02   5.000    24000.0   16.457708   
     3            1.50  0.008  1.0  1.50   5.000   409600.0   24.975379   
     4            2.00  0.009  1.0  2.00   5.000   409600.0   25.435757   
2020 1            2.00  0.002  1.0  2.00   5.000   716800.0   36.367277   
     2            2.10  0.002  1.0  2.10   7.000   512000.0  120.467532   
     3            2.50  0.002  1.0  2.50   7.000    44044.0   33.756212   
     4            3.00  0.002  1.0  3.00   8.000   100000.0   33.569093   
2021 1            3.00  0.002  1.0  3.00   8.000  2204800.0   62.641887   
     2            3.00  0.003  1.0  3.00  10.000  2204800.0   74.344228   
     3            4.00  0.010  1.0  4.00  10.000  2204800.0   96.539372   
     4            4.00  0.003  1.0  4.00  10.000  2204800.0  119.370526   
2022 1            5.00  0.003  1.0  5.00  12.610   310000.0   70.094706   
     2            6.00  0.000  2.0  6.00  20.000  3450300.0  146.290875   

                        std  
ANNO TRIMESTRE               
2017 2          1444.936110  
     3          1502.391186  
     4          1170.348889  
2018 1           220.457301  
     2           228.402898  
     3           314.737683  
     4           162.191675  
2019 1           310.757636  
     2           212.293043  
     3          1499.995346  
     4          1660.236201  
2020 1          2845.161039  
     2          3627.203825  
     3           618.434818  
     4           600.817432  
2021 1          7203.244610  
     2          7104.513966  
     3          8257.100882  
     4          9133.186780  
2022 1          1405.503956  
     2          9653.691414

In [147]:
fijo_velocidad_up.to_csv('data/internetfijo_velocidadefectiva_up.csv')

### Mediana de la velocidad efectiva de internet fijo por segmento y trimestre

In [484]:
fijo_velocidad_down_segmentos = internet_fijo.groupby(['ANNO', 'TRIMESTRE', 'SEGMENTO']).VELOCIDAD_EFECTIVA_DOWNSTREAM.median().reset_index().pivot_table(index=['ANNO', 'TRIMESTRE'], columns='SEGMENTO', values='VELOCIDAD_EFECTIVA_DOWNSTREAM')

In [485]:
fijo_velocidad_up_segmentos = internet_fijo.groupby(['ANNO', 'TRIMESTRE', 'SEGMENTO']).VELOCIDAD_EFECTIVA_UPSTREAM.median().reset_index().pivot_table(index=['ANNO', 'TRIMESTRE'], columns='SEGMENTO', values='VELOCIDAD_EFECTIVA_UPSTREAM')

In [486]:
fijo_velocidad_down_segmentos

SEGMENTO        Corporativo  Residencial - Estrato 1  Residencial - Estrato 2  \
ANNO TRIMESTRE                                                                  
2017 2                5.859                     3.00                     4.00   
     3                6.000                     3.50                     4.00   
     4                6.000                     4.00                     4.00   
2018 1                6.000                     4.00                     4.00   
     2                6.000                     4.00                     4.10   
     3                6.000                     4.10                     5.00   
     4                7.000                     5.00                     5.00   
2019 1                8.000                     5.00                     5.12   
     2                8.000                     6.00                     6.00   
     3                8.000                     6.00                     6.00   
     4                8.190                     8.00                     9.00   
2020 1               10.000                    10.00                    10.00   
     2               10.000                     8.00                     8.00   
     3               10.000                    10.00                    10.00   
     4               10.000                    10.00                    10.00   
2021 1               11.000                    10.00                    10.00   
     2               12.000                    10.00                    10.00   
     3               12.000                    10.00                    10.00   
     4               14.000                    10.00                    10.00   
2022 1               15.000                    10.00                    11.00   
     2               20.000                    10.24                    12.00   

SEGMENTO        Residencial - Estrato 3  Residencial - Estrato 4  \
ANNO TRIMESTRE                                                     
2017 2                             5.00                     5.00   
     3                             5.00                     5.00   
     4                             5.00                     5.12   
2018 1                             5.00                     6.00   
     2                             5.12                     6.00   
     3                             6.00                     8.00   
     4                             6.00                     8.00   
2019 1                             6.00                    10.00   
     2                             8.00                    10.00   
     3                             6.00                    10.00   
     4                             8.00                    10.00   
2020 1                             8.00                    10.00   
     2                            10.00                    10.24   
     3                            10.00                    12.00   
     4                            10.00                    15.00   
2021 1                            10.00                    15.00   
     2                            10.00                    16.00   
     3                            11.00                    20.00   
     4                            12.00                    25.00   
2022 1                            15.00                    30.00   
     2                            16.00                    50.00   

SEGMENTO        Residencial - Estrato 5  Residencial - Estrato 6  \
ANNO TRIMESTRE                                                     
2017 2                              6.0                     6.00   
     3                              6.0                     6.00   
     4                              6.0                     6.00   
2018 1                              6.0                     7.00   
     2                              7.0                     8.00   
     3                              8.0                    10.00   
     4                     

In [489]:
fijo_velocidad_down_segmentos.to_csv('data/internetfijo_velocidadefectiva_down_segmentos.csv')
fijo_velocidad_up_segmentos.to_csv('data/internetfijo_velocidadefectiva_up_segmentos.csv')

### Diferencia de la mediana de velocidad de internet fijo entre 2017 y 2022

In [491]:
fijo_velocidad_diferencias = pd.concat([
    (fijo_velocidad_down_segmentos.iloc[-1] - fijo_velocidad_down_segmentos.iloc[0]).rename('down'),
    (fijo_velocidad_up_segmentos.iloc[-1] - fijo_velocidad_up_segmentos.iloc[0]).rename('up')
], axis=1)

In [492]:
fijo_velocidad_diferencias

,down,up
SEGMENTO,,
Corporativo,14.141,8.0
Residencial - Estrato 1,7.240,3.0
Residencial - Estrato 2,8.000,4.0
Residencial - Estrato 3,11.000,5.0
Residencial - Estrato 4,45.000,9.0
Residencial - Estrato 5,54.000,9.0
Residencial - Estrato 6,74.000,9.0
Sin estratificar,7.000,5.0
Uso propio interno del operador,NaN,NaN


In [499]:
fijo_velocidad_diferencias.to_csv('data/internetfijo_velocidadefectiva_segmentos_cambio_2017_2022.csv')

In [495]:
fijo_velocidad_diferencias.iloc[1:4].mean()

down    8.746667
up      4.000000
dtype: float64

In [498]:
fijo_velocidad_diferencias.iloc[4:7].mean()

down    57.666667
up       9.000000
dtype: float64

### Número y porcentaje de accesos a internet fijo por tecnología y trimestre

In [502]:
fijo_tec = aggregate_by_col(internet_fijo, 'TECNOLOGIA')

In [508]:
fijo_tec.to_csv('data/internetfijo_accesos_por_tecnologia.csv')

In [506]:
(fijo_tec.div(fijo_tec.sum(axis=1), axis=0) * 100).astype(int)

TECNOLOGIA      Cable  Fiber to the antenna (FTTA)  \
ANNO TRIMESTRE                                       
2017 2             38                            0   
     3             39                            0   
     4             38                            0   
2018 1             38                            0   
     2             37                            0   
     3             37                            0   
     4             39                            0   
2019 1             37                            0   
     2             38                            0   
     3             37                            0   
     4             39                            0   
2020 1             38                            0   
     2             38                            0   
     3             39                            0   
     4             39                            0   
2021 1             39                            0   
     2             38                            0   
     3             37                            0   
     4             38                            0   
2022 1             37                            0   
     2             38                            0   

TECNOLOGIA      Fiber to the building o fiber to the basement (FTTB)  \
ANNO TRIMESTRE                                                         
2017 2                                                             0   
     3                                                             0   
     4                                                             0   
2018 1                                                             0   
     2                                                             0   
     3                                                             0   
     4                                                             0   
2019 1                                                             0   
     2                                                             0   
     3                                                             0   
     4                                                             0   
2020 1                                                             0   
     2                                                             0   
     3                                                             0   
     4                                                             0   
2021 1                                                             0   
     2                                                             0   
     3                                                             0   
     4                                                             0   
2022 1                                                             0   
     2                                                             0   

TECNOLOGIA      Fiber to the cabinet (FTTC)  Fiber to the home (FTTH)  \
ANNO TRIMESTRE                                                          
2017 2                                    0                         4   
     3                                    1                         4   
     4                                    3                         4   
2018 1                                    3                         5   
     2                                    3                         5   
     3                                    3                         6   
     4                                    3                         6   
2019 1                                    3                         6   
     2                                    3                         7   
     3                                    3                         8   
     4                                    1                         9   
2020 1                                    1                         9   
     2                                    1                        10   
     

In [510]:
fijo_tec_porcentaje = fijo_tec.div(fijo_tec.sum(axis=1), axis=0)

In [518]:
(fijo_tec_porcentaje * 100)

TECNOLOGIA          Cable  Fiber to the antenna (FTTA)  \
ANNO TRIMESTRE                                           
2017 2          38.655524                     0.007201   
     3          39.049217                     0.007097   
     4          38.466988                     0.007681   
2018 1          38.484290                     0.006744   
     2          37.292471                     0.006719   
     3          37.324720                     0.006637   
     4          39.306967                     0.006640   
2019 1          37.947423                     0.006734   
     2          38.082699                     0.006613   
     3          37.962445                     0.001469   
     4          39.469982                     0.000014   
2020 1          38.812131                     0.002680   
     2          38.749240                     0.006181   
     3          39.234029                     0.018445   
     4          39.436428                     0.004059   
2021 1          39.375483                     0.004405   
     2          38.857308                     0.004643   
     3          37.958772                     0.004867   
     4          38.101989                     0.004500   
2022 1          37.862488                     0.001299   
     2          38.283159                     0.000000   

TECNOLOGIA      Fiber to the building o fiber to the basement (FTTB)  \
ANNO TRIMESTRE                                                         
2017 2                                                      0.050293   
     3                                                      0.053735   
     4                                                      0.057469   
2018 1                                                      0.064989   
     2                                                      0.067369   
     3                                                      0.065660   
     4                                                      0.070534   
2019 1                                                      0.072153   
     2                                                      0.071783   
     3                                                      0.065736   
     4                                                      0.070190   
2020 1                                                      0.069696   
     2                                                      0.073400   
     3                                                      0.071559   
     4                                                      0.069850   
2021 1                                                      0.069223   
     2                                                      0.076058   
     3                                                      0.083584   
     4                                                      0.101304   
2022 1                                                      0.077204   
     2                                                      0.077182   

TECNOLOGIA      Fiber to the cabinet (FTTC)  Fiber to the home (FTTH)  \
ANNO TRIMESTRE                                                          
2017 2                             0.791728                  4.148880   
     3                             1.068453                  4.576192   
     4                             3.652652                  4.802103   
2018 1                             3.810394                  5.293265   
     2                             3.900625                  5.696781   
     3                             3.521121                  6.152356   
     4                             3.452866                  6.508036   
2019 1                             3.458663                  6.832675   
     2                             3.497472                  7.493216   
     3                             3.247427                  8.503029   
     4                             1.587639                  9.226155   
2020 1                             1.791760                

In [517]:
fijo_tec_porcentaje.to_csv('data/internetfijo_accesos_por_tecnologia_porcentaje.csv')

In [516]:
fijo_tec_porcentaje[['Hybrid Fiber Coaxial (HFC)', 'Fiber to the home (FTTH)']].sum(axis=1)

ANNO  TRIMESTRE
2017  2            0.183863
      3            0.189948
      4            0.201496
2018  1            0.209774
      2            0.231521
      3            0.243822
      4            0.254104
2019  1            0.263478
      2            0.273573
      3            0.288518
      4            0.293194
2020  1            0.307314
      2            0.317119
      3            0.325379
      4            0.337913
2021  1            0.349474
      2            0.368952
      3            0.388336
      4            0.395891
2022  1            0.412618
      2            0.432875
dtype: float64

In [563]:
fijo_ingresos = pd.read_csv('data/sources/internetfijo_ingresos.csv', sep=';', decimal=',')

In [564]:
fijo_ingresos

,ANNO,TRIMESTRE,ID_EMPRESA,EMPRESA,ID_SERVICIO,SERVICIO,ID_CONCEPTO,CONCEPTO,INGRESOS
0,2012,1,900022836,@NY PHONE S A ESP,101,Acceso fijo a Internet,0,NaN,4.605310e+07
1,2012,1,830016046,AVANTEL S.A.S,101,Acceso fijo a Internet,0,NaN,1.981290e+08
2,2012,1,830119051,AXESAT S.A.,101,Acceso fijo a Internet,0,NaN,2.534731e+09
3,2012,1,900258177,BRASIL TELECOM DE COLOMBIA EMPRESA UNIPERSONAL,101,Acceso fijo a Internet,0,NaN,2.082415e+08
4,2012,1,900047715,CABLE & TV YOPAL SAS,101,Acceso fijo a Internet,0,NaN,5.742000e+07
...,...,...,...,...,...,...,...,...,...
12244,2022,2,2800000008,GROUP TELECOMUNICACIONES P&C SAS,101,Acceso fijo a Internet,0,NaN,2.595916e+08
12245,2022,2,2800000009,SUSETELCO S.A.S,101,Acceso fijo a Internet,0,NaN,1.752497e+08
12246,2022,2,2800000010,REDES INTEGRALES DE TELECOMUNICACIONES AyG SAS,101,Acceso fijo a Internet,0,NaN,4.353576e+08
12247,2022,2,2800000011,TV SUR LTDA,101,Acceso fijo a Internet,0,NaN,3.051694e+07


In [565]:
fijo_ingresos = fijo_ingresos[fijo_ingresos[['ANNO', 'TRIMESTRE']].apply(lambda x: ''.join([str(i) for i in x]), axis=1) > '20171']

In [567]:
fijo_ingresos

,ANNO,TRIMESTRE,ID_EMPRESA,EMPRESA,ID_SERVICIO,SERVICIO,ID_CONCEPTO,CONCEPTO,INGRESOS
2105,2017,2,800196048,TEKTRON COMUNICACIONES LTDA,101,Acceso fijo a Internet,101,Ingresos operacionales,0.000000e+00
2106,2017,2,900182763,SERVICIOS PROFESIONALES EN NETWORKING LIMITADA,101,Acceso fijo a Internet,101,Ingresos operacionales,8.693494e+07
2107,2017,2,900797091,EURONA TELECOM COLOMBIA S A S,101,Acceso fijo a Internet,101,Ingresos operacionales,2.731000e+04
2108,2017,2,900459427,ACCESS POINT DE COLOMBIA S.A.S.,101,Acceso fijo a Internet,101,Ingresos operacionales,5.027513e+06
2109,2017,2,890399003,EMPRESAS MUNICIPALES DE CALI EICE E.S.P,101,Acceso fijo a Internet,101,Ingresos operacionales,1.909190e+10
...,...,...,...,...,...,...,...,...,...
12244,2022,2,2800000008,GROUP TELECOMUNICACIONES P&C SAS,101,Acceso fijo a Internet,0,NaN,2.595916e+08
12245,2022,2,2800000009,SUSETELCO S.A.S,101,Acceso fijo a Internet,0,NaN,1.752497e+08
12246,2022,2,2800000010,REDES INTEGRALES DE TELECOMUNICACIONES AyG SAS,101,Acceso fijo a Internet,0,NaN,4.353576e+08
12247,2022,2,2800000011,TV SUR LTDA,101,Acceso fijo a Internet,0,NaN,3.051694e+07


In [617]:
fijo_ingresos_total = pd.DataFrame(fijo_ingresos.groupby(['ANNO', 'TRIMESTRE']).INGRESOS.sum().rename('ingresos'))

fijo_ingresos_total['ingresos crecimiento trimestral'] = fijo_ingresos_total.div(fijo_ingresos_total.shift())
fijo_ingresos_total['ingresos respecto a 2017-2T'] = fijo_ingresos_total.ingresos / fijo_ingresos_total.loc[(2017,2), 'ingresos']
fijo_ingresos_total['ingresos por acceso'] = fijo_ingresos_total.ingresos / fijo_accesos
fijo_ingresos_total['por acceso crecimiento trimestral'] = fijo_ingresos_total['ingresos por acceso'].div(fijo_ingresos_total['ingresos por acceso'].shift())
fijo_ingresos_total['por acceso respecto a 2017-2T'] = fijo_ingresos_total['ingresos por acceso'].div(fijo_ingresos_total.loc[(2017,2), 'ingresos por acceso'])

In [618]:
fijo_ingresos_total

ingresos  ingresos crecimiento trimestral  \
ANNO TRIMESTRE                                                  
2017 2          1.002747e+12                              NaN   
     3          1.043035e+12                         1.040177   
     4          1.072036e+12                         1.027805   
2018 1          1.086901e+12                         1.013866   
     2          1.167925e+12                         1.074546   
     3          1.174300e+12                         1.005458   
     4          1.200726e+12                         1.022504   
2019 1          1.181114e+12                         0.983666   
     2          1.206434e+12                         1.021437   
     3          1.222065e+12                         1.012957   
     4          1.268218e+12                         1.037767   
2020 1          1.268778e+12                         1.000441   
     2          1.295297e+12                         1.020901   
     3          1.374773e+12                         1.061358   
     4          1.458201e+12                         1.060685   
2021 1          1.498010e+12                         1.027300   
     2          1.529883e+12                         1.021277   
     3          1.554586e+12                         1.016147   
     4          1.607788e+12                         1.034223   
2022 1          1.591811e+12                         0.990063   
     2          1.620234e+12                         1.017856   

                ingresos respecto a 2017-2T  ingresos por acceso  \
ANNO TRIMESTRE                                                     
2017 2                             1.000000        161898.627878   
     3                             1.040177        165969.251305   
     4                             1.069100        168746.314066   
2018 1                             1.083923        167742.139411   
     2                             1.164725        177532.222523   
     3                             1.171083        176319.519920   
     4                             1.197437        178374.012716   
2019 1                             1.177878        173671.131804   
     2                             1.203129        174565.177770   
     3                             1.218717        174258.678882   
     4                             1.264744        181629.619184   
2020 1                             1.265302        177071.172952   
     2                             1.291749        172926.346490   
     3                             1.371007        178576.285580   
     4                             1.454207        186105.605432   
2021 1                             1.493907        184317.519695   
     2                             1.525692        184492.594818   
     3                             1.550327        186371.824133   
     4                             1.603384        190386.233972   
2022 1                             1.587450        186259.275701   
     2                             1.615796        191007.223571   

                por acceso crecimiento trimestral  \
ANNO TRIMESTRE                                      
2017 2                                        NaN   
     3                                   1.025143   
     4                                   1.016732   
2018 1                                   0.994049   
     2                                   1.058364   
     3                                   0.993169   
     4                                   1.011652   
2019 1                                   0.973635   
     2                                   1.005148   
     3                                   0.998244   
     4                                   1.042299   
2020 1                                   0.974903   
     2                                   0.976592   
     3                                   1.032673   
     4                                   1.042163   
2021 1                                   0.990392   
     

In [619]:
fijo_ingresos_total.to_csv('data/internetfijo_ingresos.csv')

### Penetración de internet fijo por municipio y trimestre

In [629]:
poblacion_mun = pd.read_excel('data/sources/anexo-proyecciones-hogares-dptal-2018-2050-mpal-2018-2035.xlsx', sheet_name='Proyecciones Hogares mpio')

/home/m/.virtualenvs/d/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [672]:
poblacion_mun = poblacion_mun[poblacion_mun['Área'] == 'Total']
poblacion_mun = poblacion_mun.set_index('Código Municipio')[poblacion_mun.columns[5:10]].stack()
poblacion_mun = poblacion_mun.reset_index(name='hogares')
poblacion_mun.columns = ['ID_MUNICIPIO', 'ANNO', 'hogares']
poblacion_mun.ANNO = poblacion_mun.ANNO.astype(int)

In [664]:
poblacion_mun

,ID_MUNICIPIO,ANNO,hogares
0,5001,2018,813769
1,5001,2019,846522
2,5001,2020,878480
3,5001,2021,908775
4,5001,2022,939966
...,...,...,...
5605,99773,2018,16648
5606,99773,2019,17917
5607,99773,2020,18987
5608,99773,2021,19655


In [661]:
fijo_mun = aggregate_by_col(internet_fijo, 'ID_MUNICIPIO')
fijo_mun = fijo_mun.stack().reset_index(name='accesos')
fijo_mun = fijo_mun[fijo_mun.ANNO >= 2018]

In [662]:
fijo_mun

,ANNO,TRIMESTRE,ID_MUNICIPIO,accesos
3366,2018,1,0,0
3367,2018,1,5001,574317
3368,2018,1,5002,985
3369,2018,1,5004,73
3370,2018,1,5021,302
...,...,...,...,...
23557,2022,2,97889,1
23558,2022,2,99001,126
23559,2022,2,99524,18
23560,2022,2,99624,10


In [684]:
mun_dict = internet_fijo.groupby('ID_MUNICIPIO').MUNICIPIO.first().to_dict()
mundep_dict = internet_fijo.groupby('ID_MUNICIPIO').DEPARTAMENTO.first().to_dict()

In [675]:
fijo_mun = pd.merge(fijo_mun, poblacion_mun, on=['ANNO', 'ID_MUNICIPIO'])

In [688]:
fijo_mun.insert(3, 'municipio', fijo_mun.ID_MUNICIPIO.map(mun_dict))
fijo_mun.insert(3, 'departamento', fijo_mun.ID_MUNICIPIO.map(mundep_dict))
fijo_mun['penetracion'] = fijo_mun.accesos / fijo_mun.hogares

In [689]:
fijo_mun

,ANNO,TRIMESTRE,ID_MUNICIPIO,departamento,municipio,accesos,hogares,penetracion
0,2018,1,5001,ANTIOQUIA,MEDELLÍN,574317,813769,0.705749
1,2018,2,5001,ANTIOQUIA,MEDELLÍN,578335,813769,0.710687
2,2018,3,5001,ANTIOQUIA,MEDELLÍN,586446,813769,0.720654
3,2018,4,5001,ANTIOQUIA,MEDELLÍN,592660,813769,0.728290
4,2018,1,5002,ANTIOQUIA,ABEJORRAL,985,6751,0.145904
...,...,...,...,...,...,...,...,...
20155,2022,2,99524,VICHADA,LA PRIMAVERA,18,3110,0.005788
20156,2022,1,99624,VICHADA,SANTA ROSALÍA,12,1281,0.009368
20157,2022,2,99624,VICHADA,SANTA ROSALÍA,10,1281,0.007806
20158,2022,1,99773,VICHADA,CUMARIBO,54,20274,0.002664


In [692]:
fijo_mun = fijo_mun.pivot_table(index=['ID_MUNICIPIO', 'departamento', 'municipio'], columns=['ANNO', 'TRIMESTRE'], values='penetracion')

In [693]:
fijo_mun

ANNO                                          2018                      \
TRIMESTRE                                        1         2         3   
ID_MUNICIPIO departamento municipio                                      
5001         ANTIOQUIA    MEDELLÍN        0.705749  0.710687  0.720654   
5002         ANTIOQUIA    ABEJORRAL       0.145904  0.151533  0.154940   
5004         ANTIOQUIA    ABRIAQUÍ        0.078834  0.080994  0.078834   
5021         ANTIOQUIA    ALEJANDRÍA      0.185618  0.189305  0.197910   
5030         ANTIOQUIA    AMAGÁ           0.278812  0.288270  0.294027   
...                                            ...       ...       ...   
97889        VAUPÉS       YAVARATÉ        0.000000  0.000000  0.000000   
99001        VICHADA      PUERTO CARREÑO  0.222666  0.142423  0.125000   
99524        VICHADA      LA PRIMAVERA    0.013005  0.013005  0.011636   
99624        VICHADA      SANTA ROSALÍA   0.011775  0.011775  0.010870   
99773        VICHADA      CUMARIBO        0.004565  0.004505  0.004505   

ANNO                                                    2019            \
TRIMESTRE                                        4         1         2   
ID_MUNICIPIO departamento municipio                                      
5001         ANTIOQUIA    MEDELLÍN        0.728290  0.709754  0.716085   
5002         ANTIOQUIA    ABEJORRAL       0.153607  0.157116  0.160047   
5004         ANTIOQUIA    ABRIAQUÍ        0.076674  0.086681  0.086681   
5021         ANTIOQUIA    ALEJANDRÍA      0.194837  0.190706  0.194931   
5030         ANTIOQUIA    AMAGÁ           0.308934  0.309720  0.315690   
...                                            ...       ...       ...   
97889        VAUPÉS       YAVARATÉ        0.000000  0.000000  0.000000   
99001        VICHADA      PUERTO CARREÑO  0.130754  0.182778  0.180698   
99524        VICHADA      LA PRIMAVERA    0.011636  0.021013  0.017224   
99624        VICHADA      SANTA ROSALÍA   0.010870  0.010408  0.009540   
99773        VICHADA      CUMARIBO        0.004325  0.005972  0.005637   

ANNO                                                              2020  \
TRIMESTRE                                        3         4         1   
ID_MUNICIPIO departamento municipio                                      
5001         ANTIOQUIA    MEDELLÍN        0.722828  0.731025  0.719525   
5002         ANTIOQUIA    ABEJORRAL       0.161513  0.157409  0.154855   
5004         ANTIOQUIA    ABRIAQUÍ        0.089852  0.087738  0.099174   
5021         ANTIOQUIA    ALEJANDRÍA      0.197345  0.182861  0.170012   
5030         ANTIOQUIA    AMAGÁ           0.321660  0.316486  0.297753   
...                                            ...       ...       ...   
97889        VAUPÉS       YAVARATÉ        0.008333  0.008333  0.008000   
99001        VICHADA      PUERTO CARREÑO  0.198464  0.174616  0.204082   
99524        VICHADA      LA PRIMAVERA    0.013779  0.008267  0.006511   
99624        VICHADA      SANTA ROSALÍA   0.008673  0.013877  0.011657   
99773        VICHADA      CUMARIBO        0.002288  0.001898  0.001896   

ANNO                                                                    \
TRIMESTRE                                        2         3         4   
ID_MUNICIPIO departamento municipio                                      
5001         ANTIOQUIA    MEDELLÍN        0.746222  0.750668  0.760814   
5002         ANTIOQUIA    ABEJORRAL       2.326131  3.077643  3.563094   
5004         ANTIOQUIA    ABRIAQUÍ        0.128099  0.139463  0.138430   
5021         ANTIOQUIA    ALEJANDRÍA      0.162338  0.157025  0.132231   
5030         ANTIOQUIA    AMAGÁ           0.292161  0.299875  0.288111   
...                                            ...       ...       ...   
97889        VAUPÉS       YAVARATÉ        0.008000  0.000000  0.008000   
99001        VICHADA      PUERTO CARREÑO  0.214839  0.221009  0.211359   
99524        VICHADA      LA PRIMAVERA    0.006511  0.007197  0.006511   
9

In [694]:
fijo_mun.to_csv('data/internetfijo_penetracion_por_municipio.csv')

### Número de municipios en distintos rangos de porcentaje de penetración

In [ ]:
fijo_rangos_penetracion = pd.concat([pd.cut(fijo_mun[c], bins=[-1, 0, .05, .1, .15, .2, .5, 3], labels=['0', '0.1 a 5%', '5 a 10%', '10 a 15%', '15 a 20%', 'de 20 a 50%', 'más de 50%']).value_counts() for c in fijo_mun.columns], axis=1).T[['0', '0.1 a 5%', '5 a 10%', '10 a 15%', '15 a 20%', 'de 20 a 50%', 'más de 50%']]

In [726]:
fijo_rangos_penetracion

0  0.1 a 5%  5 a 10%  10 a 15%  15 a 20%  de 20 a 50%  más de 50%
2018 1  15       434      266       133        77          151          44
     2  10       428      264       138        86          149          45
     3   9       457      248       130        82          147          47
     4  13       442      254       128        81          150          52
2019 1  10       443      254       121        84          161          47
     2   6       426      270       123        81          165          49
     3   5       444      251       118        78          173          51
     4   5       608      131        97        64          163          52
2020 1   6       589      128       112        68          165          52
     2   7       549      151       111        62          175          65
     3   8       525      151       108        72          185          70
     4   2       510      152       116        76          189          74
2021 1   3       459      162       140        73          201          82
     2   3       417      175       128        92          211          94
     3   2       398      175       136        94          213         102
     4   3       380      191       143        85          219          99
2022 1   2       400      207       125        86          210          90
     2   5       446      186       111        77          207          88

In [733]:
fijo_rangos_penetracion.to_csv('data/internetfijo_municipios_por_rango_de_penetracion.csv')

In [732]:
pd.concat([fijo_rangos_penetracion[i].sum(axis=1) for i in [['0', '0.1 a 5%'], ['5 a 10%', '10 a 15%', '15 a 20%'], ['de 20 a 50%', 'más de 50%']]], axis=1)

0    1    2
2018 1  449  476  195
     2  438  488  194
     3  466  460  194
     4  455  463  202
2019 1  453  459  208
     2  432  474  214
     3  449  447  224
     4  613  292  215
2020 1  595  308  217
     2  556  324  240
     3  533  331  255
     4  512  344  263
2021 1  462  375  283
     2  420  395  305
     3  400  405  315
     4  383  419  318
2022 1  402  418  300
     2  451  374  295

# Internet móvil

## Fuentes

Fuentes:
- Internet móvil por cargo fijo en [postdata.gov.co](https://postdata.gov.co/dataset/suscriptores-ingresos-y-tr%C3%A1fico-de-internet-m%C3%B3vil-por-cargo-fijo)
- Internet móvil por demanda en [postdata.gov.co](https://postdata.gov.co/dataset/abonados-ingresos-y-tr%C3%A1fico-de-internet-m%C3%B3vil-por-demanda)

In [736]:
internetmovil_cargo = pd.read_csv('data/sources/internetmovil_cargofijo.csv', sep=';')

In [745]:
internetmovil_cargo.insert(0, 'mes', internetmovil_cargo[['ANNO', 'TRIMESTRE', 'MES_DEL_TRIMESTRE']].apply(lambda x: dt.date(x[0], ((x[1] - 1) * 3) + x[2], 1), axis=1))

In [746]:
internetmovil_cargo

,mes,ANNO,TRIMESTRE,MES_DEL_TRIMESTRE,ID_SEGMENTO,SEGMENTO,ID_EMPRESA,EMPRESA,ID_TERMINAL,TERMINAL,ID_TECNOLOGIA,TECNOLOGIA,CANTIDAD_SUSCRIPTORES
0,2012-01-01,2012,1,1,1,Personas,830122566,COLOMBIA TELECOMUNICACIONES S.A. ESP,1,Teléfono Móvil,1,2G,50590
1,2012-01-01,2012,1,1,1,Personas,830122566,COLOMBIA TELECOMUNICACIONES S.A. ESP,1,Teléfono Móvil,2,3G,469844
2,2012-01-01,2012,1,1,1,Personas,830122566,COLOMBIA TELECOMUNICACIONES S.A. ESP,1,Teléfono Móvil,3,4G,0
3,2012-01-01,2012,1,1,1,Personas,830122566,COLOMBIA TELECOMUNICACIONES S.A. ESP,2,Data Card,1,2G,2024
4,2012-01-01,2012,1,1,1,Personas,830122566,COLOMBIA TELECOMUNICACIONES S.A. ESP,2,Data Card,2,3G,25295
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5932,2022-06-01,2022,2,3,1,Personas,901354361,PARTNERS TELECOM COLOMBIA SAS,2,Data Card,3,4G,17884
5933,2022-06-01,2022,2,3,1,Personas,901354361,PARTNERS TELECOM COLOMBIA SAS,1,Teléfono móvil,2,3G,66116
5934,2022-06-01,2022,2,3,2,Empresas,901354361,PARTNERS TELECOM COLOMBIA SAS,1,Teléfono móvil,2,3G,531
5935,2022-06-01,2022,2,3,2,Empresas,901354361,PARTNERS TELECOM COLOMBIA SAS,1,Teléfono móvil,3,4G,5263


In [912]:
internetmovil_demanda = pd.read_csv('data/sources/internetmovil_demanda.csv', sep=';')

In [916]:
def make_date(anno, trimestre, mes_del_trimestre):
    try:
        return dt.date(anno, ((trimestre - 1) * 3) + mes_del_trimestre, 1)
    except Exception:
        print(anno, trimestre, mes_del_trimestre)

In [914]:
internetmovil_demanda.insert(0, 'mes', internetmovil_demanda[['ANNO', 'TRIMESTRE', 'MES_DEL_TRIMESTRE']].apply(lambda x: make_date(x[0], x[1], x[2]), axis=1))

In [915]:
internetmovil_demanda

,mes,ANNO,TRIMESTRE,MES_DEL_TRIMESTRE,ID_EMPRESA,EMPRESA,ID_MODALIDAD_PAGO,MODALIDAD_PAGO,ID_TERMINAL,TERMINAL,ID_TECNOLOGIA,TECNOLOGIA,CANTIDAD_ABONADOS
0,2012-01-01,2012,1,1,900367669,UFF MOVIL SAS,PRE,Prepago,1,Teléfono móvil,2,3G,14899
1,2012-01-01,2012,1,1,830122566,COLOMBIA TELECOMUNICACIONES S.A. ESP,POS,Pospago,1,Teléfono móvil,1,2G,54637
2,2012-01-01,2012,1,1,830122566,COLOMBIA TELECOMUNICACIONES S.A. ESP,POS,Pospago,1,Teléfono móvil,2,3G,172006
3,2012-01-01,2012,1,1,830122566,COLOMBIA TELECOMUNICACIONES S.A. ESP,POS,Pospago,1,Teléfono móvil,3,4G,0
4,2012-01-01,2012,1,1,830122566,COLOMBIA TELECOMUNICACIONES S.A. ESP,POS,Pospago,2,Data Card,1,2G,2063
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5809,2022-06-01,2022,2,3,900389508,ALMACENES EXITO INVERSIONES S.A.S.,PRE,Prepago,1,Teléfono móvil,1,2G,980
5810,2022-06-01,2022,2,3,830130125,CELLVOZ COLOMBIA SERVICIOS INTEGRALES S A E S P,PRE,Prepago,1,Teléfono móvil,3,4G,123
5811,2022-06-01,2022,2,3,830130125,CELLVOZ COLOMBIA SERVICIOS INTEGRALES S A E S P,PRE,Prepago,1,Teléfono móvil,2,3G,6
5812,2022-06-01,2022,2,3,830114921,COLOMBIA MOVIL S.A ESP,PRE,Prepago,1,Teléfono móvil,3,4G,3230166


## Ideas

### Número de accesos a internet móvil

In [806]:
movil_usuarios = pd.concat([i[0].groupby('mes')[i[1]].sum().rename(i[2]) for i in zip([internetmovil_cargo, internetmovil_demanda], ['CANTIDAD_SUSCRIPTORES', 'CANTIDAD_ABONADOS'], ['cargo_fijo', 'demanda'])], axis=1)

In [807]:
movil_usuarios['total'] = movil_usuarios.cargo_fijo + movil_usuarios.demanda

In [808]:
for i in ['cargo_fijo', 'demanda']:
    movil_usuarios['{}_porcentaje'.format(i)] = movil_usuarios[i] / movil_usuarios.total

In [809]:
for i in ['cargo_fijo', 'demanda']:
    movil_usuarios['{}_crecimiento'.format(i)] = movil_usuarios[i].diff()

In [810]:
movil_usuarios['total_crecimiento'] = movil_usuarios.total.diff()

In [811]:
movil_usuarios

,cargo_fijo,demanda,total,cargo_fijo_porcentaje,demanda_porcentaje,cargo_fijo_crecimiento,demanda_crecimiento,total_crecimiento
mes,,,,,,,,
2012-01-01,1640747,2079780,3720527,0.440999,0.559001,NaN,NaN,NaN
2012-02-01,1735952,1757384,3493336,0.496932,0.503068,95205.0,-322396.0,-227191.0
2012-03-01,2400785,2422074,4822859,0.497793,0.502207,664833.0,664690.0,1329523.0
2012-04-01,2445381,2261906,4707287,0.519488,0.480512,44596.0,-160168.0,-115572.0
2012-05-01,2545099,2533648,5078747,0.501127,0.498873,99718.0,271742.0,371460.0
...,...,...,...,...,...,...,...,...
2022-02-01,16805536,20477148,37282684,0.450760,0.549240,242693.0,-227232.0,15461.0
2022-03-01,17037587,20639549,37677136,0.452200,0.547800,232051.0,162401.0,394452.0
2022-04-01,17297649,20107468,37405117,0.462441,0.537559,260062.0,-532081.0,-272019.0


In [812]:
movil_usuarios.to_csv('data/internetmovil_accesos.csv')

### Concentración de suscriptores y abonados en la empresa líder

In [824]:
movil_empresas_cargo, movil_empresas_demanda = [
    i[0].groupby(['mes', 'EMPRESA'], as_index=False)[i[1]].sum().pivot_table(index='mes', columns='EMPRESA', values=i[1]).fillna(0).astype(int)
    for i in zip([internetmovil_cargo, internetmovil_demanda], ['CANTIDAD_SUSCRIPTORES', 'CANTIDAD_ABONADOS'])
]

In [832]:
movil_empresas_cargo, movil_empresas_demanda = [i[i.iloc[-1].sort_values(ascending=False).index] for i in [movil_empresas_cargo, movil_empresas_demanda]]

In [857]:
movil_empresas_cargo

EMPRESA,COMUNICACION CELULAR S A COMCEL S A,COLOMBIA TELECOMUNICACIONES S.A. E.S.P.,COLOMBIA MOVIL S.A ESP,PARTNERS TELECOM COLOMBIA SAS,EMPRESA DE TELECOMUNICACIONES DE BOGOTA S.A. ESP,AVANTEL S.A.S,ALMACENES EXITO INVERSIONES S.A.S.,AVANTEL S.A.S.,COLOMBIA MOVIL S.A. E.S.P.,COLOMBIA TELECOMUNICACIONES S.A. ESP,EMPRESA DE TELECOMUNICACIONES DE BOGOTÁ S.A. ESP.,SETROC MOBILE GROUP SAS,UNE EPM TELECOMUNICACIONES S.A.,UNE EPM TELECOMUNICACIONES S.A. E.S.P. - UNE EPM TELCO S.A.,VIRGIN MOBILE COLOMBIA S.A.S.
mes,,,,,,,,,,,,,,,
2012-01-01,863043,0,0,0,0,0,0,0,0,777704,0,0,0,0,0
2012-02-01,887936,0,0,0,0,0,0,0,0,848016,0,0,0,0,0
2012-03-01,923904,0,0,0,0,0,0,0,529485,907749,39647,0,0,0,0
2012-04-01,944425,0,0,0,0,0,0,0,542320,958636,0,0,0,0,0
2012-05-01,987447,0,0,0,0,0,0,0,555601,1002051,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-01,8496799,4994079,2395621,802688,93310,23039,0,0,0,0,0,0,0,0,0
2022-03-01,8604864,5043859,2446842,833479,91015,17528,0,0,0,0,0,0,0,0,0
2022-04-01,8670140,5086668,2576173,861030,89926,13712,0,0,0,0,0,0,0,0,0


In [858]:
movil_empresas_cargo.to_csv('data/internetmovil_cargofijo_suscriptores_por_empresa.csv')
movil_empresas_demanda.to_csv('data/internetmovil_demanda_abonados_por_empresa.csv')

In [853]:
movil_concentracion = pd.concat([
    pd.DataFrame([{'mes':d, '{}_concentracion'.format(i[1]):r.div(r.sum()).max()} for d, r in i[0].iterrows()]).set_index('mes')
    for i in zip([movil_empresas_cargo, movil_empresas_demanda], ['cargo', 'demanda'])
], axis=1)

In [854]:
movil_concentracion

,cargo_concentracion,demanda_concentracion
mes,,
2012-01-01,0.526006,0.801082
2012-02-01,0.511498,0.736773
2012-03-01,0.384834,0.707120
2012-04-01,0.392019,0.573744
2012-05-01,0.393718,0.654928
...,...,...
2022-02-01,0.505595,0.545350
2022-03-01,0.505052,0.539555
2022-04-01,0.501232,0.543763


In [859]:
movil_concentracion.to_csv('data/internetmovil_participacion_empresa_lider.csv')

### Tráfico de internet móvil

In [933]:
cargofijo_trafico = pd.read_csv('data/sources/internetmovil_cargofijo_trafico.csv', sep=';', decimal=',')
demanda_trafico = pd.read_csv('data/sources/internetmovil_demanda_trafico.csv', sep=';', decimal=',')

In [934]:
demanda_trafico = demanda_trafico[demanda_trafico.MES_DEL_TRIMESTRE > 0]

In [935]:
cargofijo_trafico.insert(0, 'mes', cargofijo_trafico[['ANNO', 'TRIMESTRE', 'MES_DEL_TRIMESTRE']].apply(lambda x: make_date(x[0], x[1], x[2]), axis=1))
demanda_trafico.insert(0, 'mes', demanda_trafico[['ANNO', 'TRIMESTRE', 'MES_DEL_TRIMESTRE']].apply(lambda x: make_date(x[0], x[1], x[2]), axis=1))

In [948]:
movil_trafico = pd.concat([dfi.groupby(['mes']).TRAFICO.sum().rename(name) for dfi, name in zip([cargofijo_trafico, demanda_trafico], ['cargofijo_trafico', 'demanda_trafico'])], axis=1)

In [950]:
movil_trafico['total_trafico'] = movil_trafico.sum(axis=1)

In [953]:
movil_trafico['cargofijo_porcentaje'] = movil_trafico.cargofijo_trafico / movil_trafico.total_trafico
movil_trafico['demanda_porcentaje'] = movil_trafico.demanda_trafico / movil_trafico.total_trafico

In [955]:
movil_trafico['cargofijo_por_acceso'] = movil_trafico.cargofijo_trafico / movil_usuarios.cargo_fijo
movil_trafico['demanda_por_acceso'] = movil_trafico.demanda_trafico / movil_usuarios.demanda

In [956]:
movil_trafico

,cargofijo_trafico,demanda_trafico,total_trafico,cargofijo_porcentaje,demanda_porcentaje,cargofijo_por_acceso,demanda_por_acceso
mes,,,,,,,
2012-01-01,2.383451e+09,2.110121e+08,2.594463e+09,0.918668,0.081332,1452.662154,101.458859
2012-02-01,2.068031e+09,2.395105e+08,2.307542e+09,0.896205,0.103795,1191.295128,136.288064
2012-03-01,3.255906e+09,3.350371e+08,3.590943e+09,0.906699,0.093301,1356.183792,138.326554
2012-04-01,3.247345e+09,3.925500e+08,3.639895e+09,0.892153,0.107847,1327.950525,173.548313
2012-05-01,3.447005e+09,4.550212e+08,3.902026e+09,0.883388,0.116612,1354.369681,179.591343
...,...,...,...,...,...,...,...
2022-02-01,1.507912e+11,4.891916e+10,1.997104e+11,0.755049,0.244951,8972.709962,2388.963541
2022-03-01,1.763561e+11,5.652978e+10,2.328859e+11,0.757264,0.242736,10351.000964,2738.905794
2022-04-01,1.740850e+11,5.299738e+10,2.270823e+11,0.766616,0.233384,10064.081670,2635.706535


In [959]:
movil_trafico.to_csv('data/internetmovil_trafico.csv')

### Ingresos de internet móvil

In [969]:
cargofijo_ingresos = pd.read_csv('data/sources/internetmovil_cargofijo_ingresos.csv', sep=';', decimal=',')
demanda_ingresos = pd.read_csv('data/sources/internetmovil_demanda_ingresos.csv', sep=';', decimal=',')

In [970]:
demanda_ingresos = demanda_ingresos[demanda_ingresos.MES_DEL_TRIMESTRE > 0]

In [971]:
cargofijo_ingresos.insert(0, 'mes', cargofijo_ingresos[['ANNO', 'TRIMESTRE', 'MES_DEL_TRIMESTRE']].apply(lambda x: make_date(x[0], x[1], x[2]), axis=1))
demanda_ingresos.insert(0, 'mes', demanda_ingresos[['ANNO', 'TRIMESTRE', 'MES_DEL_TRIMESTRE']].apply(lambda x: make_date(x[0], x[1], x[2]), axis=1))

In [975]:
movil_ingresos = pd.concat([
    cargofijo_ingresos.groupby('mes').INGRESOS.sum().rename('cargofijo_ingresos'),
    demanda_ingresos.groupby('mes').INGRESOS.sum().rename('demanda_ingresos')
], axis=1)

In [977]:
movil_ingresos['total'] = movil_ingresos.sum(axis=1)

In [979]:
movil_ingresos['cargofijo_porcentaje'] = movil_ingresos.cargofijo_ingresos / movil_ingresos.total
movil_ingresos['demanda_porcentaje'] = movil_ingresos.demanda_ingresos / movil_ingresos.total

In [981]:
movil_ingresos['cargofijo_por_acceso'] = movil_ingresos.cargofijo_ingresos / movil_usuarios.cargo_fijo
movil_ingresos['demanda_por_acceso'] = movil_ingresos.demanda_ingresos / movil_usuarios.demanda

In [986]:
movil_ingresos['cargofijo_por_megabyte'] = movil_ingresos.cargofijo_ingresos / movil_trafico.cargofijo_trafico
movil_ingresos['demanda_por_megabyte'] = movil_ingresos.demanda_ingresos / movil_trafico.demanda_trafico

In [987]:
movil_ingresos

,cargofijo_ingresos,demanda_ingresos,total,cargofijo_porcentaje,demanda_porcentaje,cargofijo_por_acceso,demanda_por_acceso,cargofijo_por_megabyte,demanda_por_megabyte
mes,,,,,,,,,
2012-01-01,6.831122e+10,1.076887e+10,7.908008e+10,0.863823,0.136177,41634.218059,5177.886867,28.660634,51.034350
2012-02-01,6.628933e+10,9.564768e+09,7.585409e+10,0.873906,0.126094,38186.151717,5442.617170,32.054317,39.934658
2012-03-01,8.533419e+10,1.554518e+10,1.008794e+11,0.845903,0.154097,35544.284978,6418.128846,26.209047,46.398386
2012-04-01,8.662975e+10,1.667589e+10,1.033056e+11,0.838577,0.161423,35425.870561,7372.494250,26.677101,42.480933
2012-05-01,9.039010e+10,1.762572e+10,1.080158e+11,0.836823,0.163177,35515.358298,6956.655271,26.222795,38.736028
...,...,...,...,...,...,...,...,...,...
2022-02-01,4.887063e+11,1.897808e+11,6.784871e+11,0.720288,0.279712,29080.076563,9267.933256,3.240947,3.879479
2022-03-01,4.923532e+11,2.145358e+11,7.068890e+11,0.696507,0.303493,28898.059763,10394.402203,2.791813,3.795093
2022-04-01,4.984018e+11,2.068886e+11,7.052903e+11,0.706662,0.293338,28813.267012,10289.139979,2.862980,3.903750


In [989]:
movil_ingresos.to_csv('data/internetmovil_ingresos.csv')

# Encuesta de calidad de vida

## Fuente

Fuentes: [ECV21](https://microdatos.dane.gov.co/catalog/734/get_microdata), en particular microdatos de las preguntas a personas sobre *Tecnologías de información y comunicación* y sus características personales en *Características y composición del hogar*.

In [2]:
ecv21 = pd.read_spss('data/sources/ecv21/Tecnologias de informaciвn y comunicacion.sav')
ecv21hogar = pd.read_csv('data/sources/ecv21/Caracteristicas y composicion del hogar.csv', sep=';', decimal=',')

In [3]:
ecv21_dict = pd.read_csv('data/sources/ecv21/ecv21_diccionario.csv')
ecv21_dict = ecv21_dict[ecv21_dict.columna.notna()][['columna', 'texto']]

In [4]:
ecv21_dict[ecv21_dict.texto.str.lower().str.contains('internet')]

,columna,texto
47,P1084,3. ¿Con qué frecuencia utiliza ... Internet (en cualquier lugar y desde cualquier dispositivo)?
53,P765,4. Cuáles de los siguientes dispositivos utiliza ... para acceder a Internet:
62,P1085,5. ¿En cuáles de los siguientes sitios accede ... a Internet:
67,P1085S5,5. En centros de acceso público con costo (Café Internet)?
71,P1083,6. ¿Para cuáles de los siguientes servicios o actividades utiliza ... Internet:
87,P1929,7. ¿Cuál es la principal razón por la que __________ no utiliza internet?:
123,P1080S4,4. Navegación en Internet?


In [5]:
ecv21_dict[ecv21_dict.texto.str.lower().str.contains('sexo')]

,columna,texto
149,P6020,2. Sexo:


In [6]:
ecv21 = pd.merge(ecv21, ecv21hogar, on=['DIRECTORIO', 'SECUENCIA_ENCUESTA', 'SECUENCIA_P'])

In [7]:
ecv21

,DIRECTORIO,SECUENCIA_ENCUESTA,SECUENCIA_P,ORDEN_x,FEX_C_x,P1910,P1911,P1912,P1710,P1710S1,P1710S2,P1710S3,P1710S4,P1710S5,P1710S6,P1710S7,P1710S8,P1710S9,P1084,P765,P765S1,P765S2,P765S3,P765S4,P765S5,...,P753S3,P752,P1662,P6081,P6081S1,P6087,P6083,P6083S1,P6088,P6080,P5667,P2057,P2059,P2061,P1895,P1896,P1897,P1898,P1899,P3175,P1901,P1903,P1904,P1905,P1927
0,7566839.0,1.0,1.0,1.0,500.083094,5.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,1.0,NaN,2,NaN,1.0,2,NaN,1.0,6,NaN,2.0,2.0,2.0,10.0,5.0,7.0,10.0,6.0,7.0,8.0,7.0,6.0,8.0,10.0
1,7566839.0,2.0,1.0,2.0,500.083094,3.0,5.0,5.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,3,NaN,6.0,1,1.0,NaN,6,NaN,2.0,2.0,2.0,8.0,7.0,10.0,7.0,6.0,7.0,8.0,5.0,4.0,10.0,8.0
2,7566839.0,3.0,1.0,3.0,500.083094,2.0,5.0,5.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,NaN,1.0,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,3,NaN,4.0,1,2.0,NaN,6,NaN,2.0,2.0,2.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,4.0,4.0,10.0,8.0
3,7566839.0,4.0,1.0,4.0,500.083094,1.0,5.0,5.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,NaN,1.0,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,3,NaN,4.0,1,2.0,NaN,6,NaN,2.0,2.0,2.0,8.0,8.0,8.0,8.0,8.0,8.0,7.0,3.0,4.0,10.0,10.0
4,7566840.0,1.0,1.0,1.0,457.807486,5.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,3,NaN,1.0,2,NaN,3.0,6,NaN,2.0,2.0,2.0,6.0,6.0,8.0,9.0,6.0,7.0,8.0,0.0,0.0,8.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238883,7787882.0,3.0,1.0,3.0,72.300400,5.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2,NaN,10.0,1,1.0,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
238884,7787883.0,1.0,1.0,1.0,69.792788,5.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2,NaN,9.0,2,NaN,9.0,5,NaN,1.0,NaN,1.0,8.0,8.0,8.0,9.0,8.0,8.0,8.0,4.0,2.0,9.0,7.0
238885,7787883.0,2.0,1.0,2.0,69.792788,5.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,2,NaN,9.0,1,1.0,NaN,5,NaN,1.0,NaN,1.0,8.0,7.0,7.0,9.0,8.0,8.0,7.0,3.0,2.0,8.0,7.0
238886,7787883.0,3.0,1.0,3.0,69.792788,5.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,2,NaN,1.0,2,NaN,3.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Ideas

### Acceso a internet por sexo

In [8]:
def corte_ecv(ecv, col_index, col_columns, opciones, factor_expansion='FEX_C_x'):
    dfi = ecv.groupby([col_index, col_columns], as_index=False)[factor_expansion].sum()
    for o in opciones.keys():
        dfi[o] = dfi[o].map(opciones[o])
    dfi = dfi.pivot_table(index=col_index, columns=col_columns, values=factor_expansion)
    dfi = dfi.loc[opciones[col_index].values()]
    return dfi

*¿Con qué frecuencia utiliza ... Internet (en cualquier lugar y desde cualquier dispositivo)?*

In [33]:
ecv21_sexo_accesointernet = corte_ecv(
    ecv21,
    'P1084',
    'P6020',
    {'P1084': {
        1: 'Todos los días de la semana',
        2: 'Al menos una vez a la semana, pero no cada día',
        3: 'Al menos una vez al mes, pero no cada semana',
        4: 'Al menos una vez al año, pero no cada mes',
        5: 'No utiliza internet'},
    'P6020': {
        1: 'Hombre',
        2: 'Mujer'
    }}
)

In [35]:
ecv21_sexo_accesointernet = ecv21_sexo_accesointernet.div(ecv21_sexo_accesointernet.sum())

In [36]:
ecv21_sexo_accesointernet

P6020,Hombre,Mujer
P1084,,
Todos los días de la semana,0.545477,0.565865
"Al menos una vez a la semana, pero no cada día",0.159475,0.157163
"Al menos una vez al mes, pero no cada semana",0.013383,0.013133
"Al menos una vez al año, pero no cada mes",0.002630,0.002919
No utiliza internet,0.279035,0.260920


In [37]:
ecv21_sexo_accesointernet.to_csv('data/ecv21_sexo_internet_frecuencia.csv')

*Cuáles de los siguientes dispositivos utiliza ... para acceder a Internet:*

In [54]:
ecv21[[c for c in ecv21.columns if 'P765S' in c]] = ecv21[[c for c in ecv21.columns if 'P765S' in c]].replace(np.nan, 0)

In [24]:
dispositivos_acceso = [
    {'col': 'P765S1', 'name': 'Computador de escritorio'},
    {'col': 'P765S2', 'name': 'Computador portatil'},
    {'col': 'P765S3', 'name': 'Tableta'},
    {'col': 'P765S4', 'name': 'Teléfono celular'},
    {'col': 'P765S5', 'name': 'Consolas para juegos electrónicos'},
    {'col': 'P765S6', 'name': 'Televisor inteligente'},
    {'col': 'P765S7', 'name': 'Reproductores digitales de música, video e imagen'},
    {'col': 'P765S8', 'name': 'Otro'},
]

In [27]:
dfis = [
    corte_ecv(
        ecv21,
        option['col'],
        'P6020',
        {option['col']: {
            1: option['name'],
            0: 'No'},
         'P6020': {
            1: 'Hombre',
            2: 'Mujer'
        }}
    ) for option in dispositivos_acceso
]

In [29]:
ecv21_sexo_internetacceso_dispositivo = pd.concat([
    dfi.div(dfi.sum()).loc[option['name']] for dfi, option in zip(dfis, dispositivos_acceso)
], axis=1).T

In [78]:
ecv21_sexo_internetacceso_dispositivo['mujer_menos_hombre'] = ecv21_sexo_internetacceso_dispositivo.Mujer - ecv21_sexo_internetacceso_dispositivo.Hombre
ecv21_sexo_internetacceso_dispositivo = ecv21_sexo_internetacceso_dispositivo.sort_values('mujer_menos_hombre', ascending=False)

In [79]:
ecv21_sexo_internetacceso_dispositivo

P6020,Hombre,Mujer,mujer_menos_hombre
Teléfono celular,0.686460,0.708511,0.022051
Computador portatil,0.180925,0.181750,0.000825
Otro,0.000014,0.000009,-0.000004
"Reproductores digitales de música, video e imagen",0.006869,0.005066,-0.001804
Tableta,0.026635,0.024286,-0.002350
Televisor inteligente,0.062318,0.059196,-0.003122
Computador de escritorio,0.153264,0.149292,-0.003972
Consolas para juegos electrónicos,0.011890,0.003609,-0.008281


In [80]:
ecv21_sexo_internetacceso_dispositivo.to_csv('data/ecv21_sexo_internet_dispositivo.csv')

*¿En cuáles de los siguientes sitios accede ... a Internet*

In [53]:
ecv21[[c for c in ecv21.columns if 'P1085S' in c]] = ecv21[[c for c in ecv21.columns if 'P1085S' in c]].replace(np.nan, 0)

In [48]:
lugares_acceso = [
    {'col': 'P1085S1', 'name': 'En el hogar'},
    {'col': 'P1085S2', 'name': 'En el trabajo'},
    {'col': 'P1085S3', 'name': 'En la institución educativa'},
    {'col': 'P1085S4', 'name': 'En centros de acceso público gratis'},
    {'col': 'P1085S5', 'name': 'En centros de acceso público con costo (Café Internet)'},
    {'col': 'P1085S6', 'name': 'En la vivienda de otra persona (pariente, amigo/a, vecino/a)'},
    {'col': 'P1085S7', 'name': 'En desplazamiento de un sitio a otro'},
    {'col': 'P1085S8', 'name': 'Otro sitio'}
]

In [55]:
dfis = [
    corte_ecv(
        ecv21,
        option['col'],
        'P6020',
        {option['col']: {
            1: option['name'],
            0: 'No'},
         'P6020': {
            1: 'Hombre',
            2: 'Mujer'
        }}
    ) for option in lugares_acceso
]

In [56]:
ecv21_sexo_internetacceso_lugar = pd.concat([
    dfi.div(dfi.sum()).loc[option['name']] for dfi, option in zip(dfis, lugares_acceso)
], axis=1).T

In [74]:
ecv21_sexo_internetacceso_lugar['mujer_menos_hombre'] = ecv21_sexo_internetacceso_lugar.Mujer - ecv21_sexo_internetacceso_lugar.Hombre
ecv21_sexo_internetacceso_lugar = ecv21_sexo_internetacceso_lugar.sort_values('mujer_menos_hombre', ascending=False)

In [75]:
ecv21_sexo_internetacceso_lugar

P6020,Hombre,Mujer,mujer_menos_hombre
En el hogar,0.674203,0.697474,0.023271
"En la vivienda de otra persona (pariente, amigo/a, vecino/a)",0.068738,0.070711,0.001972
Otro sitio,0.001565,0.001064,-0.000501
En centros de acceso público con costo (Café Internet),0.015457,0.014379,-0.001078
En centros de acceso público gratis,0.029249,0.027123,-0.002125
En la institución educativa,0.056532,0.051997,-0.004535
En desplazamiento de un sitio a otro,0.110261,0.090139,-0.020122
En el trabajo,0.243823,0.169309,-0.074513


In [81]:
ecv21_sexo_internetacceso_lugar.to_csv('data/ecv21_sexo_internet_lugar.csv')

*¿Para cuáles de los siguientes servicios o actividades utiliza ... Internet*

In [63]:
ecv21[[c for c in ecv21.columns if 'P1083S' in c]] = ecv21[[c for c in ecv21.columns if 'P1083S' in c]].replace(np.nan, 0)

In [64]:
actividades_acceso = [
    {'col': 'P1083S1', 'name': 'Obtener información'},
    {'col': 'P1083S2', 'name': 'Enviar o recibir correos electronicos'},
    {'col': 'P1083S3', 'name': 'Redes sociales'},
    {'col': 'P1083S4', 'name': 'Comprar/ordenar productos o servicios'},
    {'col': 'P1083S5', 'name': 'Banca electrónica y otros servicios financieros'},
    {'col': 'P1083S6', 'name': 'Educación y aprendizaje'},
    {'col': 'P1083S7', 'name': 'Trámites con entidades del gobierno (nacional, departamental o municipal)'},
    {'col': 'P1083S8', 'name': 'Descargar software, imágenes, juegos, música o jugar en línea'},
    {'col': 'P1083S9', 'name': 'Consulta de medios de comunicación (televisión, radio, periódicos, revistas, medios digitales, etc)'},
    {'col': 'P1083S10', 'name': 'Televisión, videos, películas u otro contenido audiovisual para entretenimiento'},
    {'col': 'P1083S11', 'name': 'Buscar trabajo (inscribirse en una plataforma, aplicación o convocatoria de empleo)'},
    {'col': 'P1083S12', 'name': 'Servicios en la nube (guardar información, editar archivos, etc.)'},
    {'col': 'P1083S13', 'name': 'Vender productos o servicios'},
    {'col': 'P1083S14', 'name': 'Trabajar'},
    {'col': 'P1083S15', 'name': 'Otro'}
]

In [65]:
dfis = [
    corte_ecv(
        ecv21,
        option['col'],
        'P6020',
        {option['col']: {
            1: option['name'],
            0: 'No'},
         'P6020': {
            1: 'Hombre',
            2: 'Mujer'
        }}
    ) for option in actividades_acceso
]

In [66]:
ecv21_sexo_internetacceso_actividades = pd.concat([
    dfi.div(dfi.sum()).loc[option['name']] for dfi, option in zip(dfis, actividades_acceso)
], axis=1).T

In [76]:
ecv21_sexo_internetacceso_actividades['mujer_menos_hombre'] = ecv21_sexo_internetacceso_actividades.Mujer - ecv21_sexo_internetacceso_actividades.Hombre
ecv21_sexo_internetacceso_actividades = ecv21_sexo_internetacceso_actividades.sort_values('mujer_menos_hombre', ascending=False)

In [77]:
ecv21_sexo_internetacceso_actividades

P6020,Hombre,Mujer,mujer_menos_hombre
Redes sociales,0.585811,0.616028,0.030216
Educación y aprendizaje,0.224853,0.234235,0.009382
Obtener información,0.379221,0.385494,0.006273
"Buscar trabajo (inscribirse en una plataforma, aplicación o convocatoria de empleo)",0.001162,0.001830,0.000668
"Servicios en la nube (guardar información, editar archivos, etc.)",0.044926,0.043730,-0.001196
Trabajar,0.020463,0.019251,-0.001213
Enviar o recibir correos electronicos,0.344721,0.342360,-0.002360
Vender productos o servicios,0.062703,0.059566,-0.003137
"Trámites con entidades del gobierno (nacional, departamental o municipal)",0.055539,0.050502,-0.005037
Comprar/ordenar productos o servicios,0.096077,0.088274,-0.007802


In [82]:
ecv21_sexo_internetacceso_actividades.to_csv('data/ecv21_sexo_internet_actividades.csv')